In [1]:
!pip install kfp --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

In [3]:
# %%bash

# # # Replace placeholder with your own AWS credentials
# AWS_ACCESS_KEY_ID=''
# AWS_SECRET_ACCESS_KEY=''

# !kubectl create secret generic aws-secret --from-literal=AWS_ACCESS_KEY_ID=${AWS_ACCESS_KEY_ID} --from-literal=AWS_SECRET_ACCESS_KEY=${AWS_SECRET_ACCESS_KEY}

In [4]:
def iris_comp():
    return kfp.dsl.ContainerOp(
        name='IRIS Component',
        image='seedjeffwan/iris-example:latest-1',
        command='python',
        arguments=['iris_comp.py', "--s3_bucket", "sagemaker-us-east-1-835319576252"],
        output_artifact_paths={'mlpipeline-ui-metadata': '/mlpipeline-ui-metadata.json'}
    )

In [5]:
@dsl.pipeline(
    name='IRIS Classification pipeline',
    description='IRIS Classification using LR in SKLEARN'
)
def iris_pipeline():
    iris_task = iris_comp().apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

In [6]:
kfp.compiler.Compiler().compile(iris_pipeline, 'iris-classification-pipeline.zip')

In [7]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')
my_run = client.run_pipeline(aws_experiment.id, 'iris-classification-pipeline', 
  'iris-classification-pipeline.zip')